In [1]:
import os
os.chdir("../")
%pwd

'd:\\AI\\NLP\\HandsOn\\sentiment-analysis'

In [2]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class DataIngestionConfig:
    root_dir: Path
    source_URL: str
    local_data_file: Path
    unzip_dir: Path

In [3]:
from SentiScope.constants import *
from SentiScope.utils.file_utils import *
from SentiScope.utils.config_utils import *

In [4]:
from SentiScope.constants import (CONFIG_FILE_PATH,
                                  PARAMS_FILE_PATH)
from SentiScope.utils.file_utils import (create_directories,
                                            get_size)
from SentiScope.utils.config_utils import (read_yaml,
                                           Settings,
                                           get_settings)

In [5]:
class ConfigurationManager:
    def __init__(
        self,
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH):

        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)

        create_directories([self.config.artifacts_root])

    

    def get_data_ingestion_config(self) -> DataIngestionConfig:
        config = self.config.data_ingestion

        create_directories([config.root_dir])

        data_ingestion_config = DataIngestionConfig(
            root_dir=config.root_dir,
            source_URL=config.source_URL,
            local_data_file=config.local_data_file,
            unzip_dir=config.unzip_dir 
        )

        return data_ingestion_config

In [6]:
from SentiScope.logging import logger

In [7]:
from SentiScope.utils.data_utils import download_data, unzip_data,load_data_to_dataframe

In [12]:
from datetime import datetime
from pathlib import Path
import json

class DataIngestion:
    def __init__(self, config: DataIngestionConfig):
        self.config = config
        self.data_uri = self.config.source_URL
        
        # Create timestamp directory
        self.timestamp = datetime.now().strftime('%Y%m%d_%H%M%S')
        self.root_dir = Path(self.config.root_dir)
        self.output_dir = self.root_dir / self.timestamp
        self.output_dir.mkdir(parents=True, exist_ok=True)
        
        # Update paths to use timestamp directory
        # Just use the filename from the config, not the full path
        self.local_data_file = self.output_dir / Path(self.config.local_data_file).name
        self.unzip_dir = self.output_dir / "unzipped"
        
        logger.info(f"Initialized DataIngestion with output directory: {self.output_dir}")

    def Ingest_data_uri(self):
        try:
            # Ensure parent directories exist
            self.local_data_file.parent.mkdir(parents=True, exist_ok=True)
            
            logger.info(f"Downloading data from {self.data_uri}")
            download_data(self.data_uri, str(self.local_data_file))  # Convert Path to string
            logger.info(f"Data downloaded to {self.local_data_file}")

            logger.info(f"Unzipping data to {self.unzip_dir}")
            self.unzip_dir.mkdir(parents=True, exist_ok=True)  # Ensure unzip directory exists
            unzip_data(str(self.local_data_file), str(self.unzip_dir))  # Convert Paths to strings
            logger.info(f"Data unzipped successfully")

            logger.info(f"Loading data from {self.local_data_file}")
            df = load_data_to_dataframe(str(self.local_data_file))  # Convert Path to string
            logger.info(f"Data loaded into DataFrame")

            # Save metadata about the ingestion
            metadata = {
                'timestamp': self.timestamp,
                'data_source': str(self.data_uri),
                'local_data_file': str(self.local_data_file),
                'unzip_dir': str(self.unzip_dir),
                'data_shape': df.shape if hasattr(df, 'shape') else None
            }
            
            with open(self.output_dir / 'ingestion_metadata.json', 'w') as f:
                json.dump(metadata, f, indent=4)

            return df
        
        except Exception as e:
            logger.error(f"Error during data ingestion: {e}")
            raise

In [13]:
try:
    config = ConfigurationManager()
    data_ingestion_config = config.get_data_ingestion_config()
    data_ingestion = DataIngestion(config=data_ingestion_config)
    DataFrame = data_ingestion.Ingest_data_uri()
    DataFrame.head(10)
except Exception as e:
    raise e

[2025-01-14 00:13:12,144: INFO: config_utils: yaml file: config\config.yaml loaded successfully]
[2025-01-14 00:13:12,147: INFO: config_utils: yaml file: params.yaml loaded successfully]
[2025-01-14 00:13:12,148: INFO: file_utils: created directory at: artifacts]
[2025-01-14 00:13:12,150: INFO: file_utils: created directory at: artifacts/data_ingestion]
[2025-01-14 00:13:12,156: INFO: 1303275637: Initialized DataIngestion with output directory: artifacts\data_ingestion\20250114_001312]
[2025-01-14 00:13:12,157: INFO: 1303275637: Downloading data from https://www.kaggle.com/api/v1/datasets/download/charunisa/chatgpt-sentiment-analysis]
[2025-01-14 00:13:24,274: INFO: data_utils: Data downloaded successfully to artifacts\data_ingestion\20250114_001312\data.zip]
[2025-01-14 00:13:24,276: INFO: 1303275637: Data downloaded to artifacts\data_ingestion\20250114_001312\data.zip]
[2025-01-14 00:13:24,277: INFO: 1303275637: Unzipping data to artifacts\data_ingestion\20250114_001312\unzipped]
[20

In [14]:
DataFrame.head(10)

,Unnamed: 0,tweets,labels
0,0,ChatGPT: Optimizing Language Models for Dialog...,neutral
1,1,"Try talking with ChatGPT, our new AI system wh...",good
2,2,ChatGPT: Optimizing Language Models for Dialog...,neutral
3,3,"THRILLED to share that ChatGPT, our new model ...",good
4,4,"As of 2 minutes ago, @OpenAI released their ne...",bad
5,5,"Just launched ChatGPT, our new AI system which...",good
6,6,"As of 2 minutes ago, @OpenAI released their ne...",bad
7,7,ChatGPT coming out strong refusing to help me ...,good
8,8,#0penAl just deployed a thing I've been helpin...,good
9,9,Research preview of our newest model: ChatGPT\...,neutral
